In [329]:
import pandas as pd 

In [360]:
import pandas as pd

# Carregar os dados
fighter_data = pd.read_csv('/Users/joaosequeira/ufc-stats/UFC-Web-Scraping/ufc_web_scraper/scraped_files/ufc_fighter_data.csv')
fight_stat_data = pd.read_csv('/Users/joaosequeira/ufc-stats/UFC-Web-Scraping/ufc_web_scraper/scraped_files/ufc_fight_stat_data.csv')
fight_data = pd.read_csv('/Users/joaosequeira/ufc-stats/UFC-Web-Scraping/ufc_web_scraper/scraped_files/ufc_fight_data.csv')

# Criar uma coluna de nome completo no fighter_data
fighter_data['full_name'] = fighter_data['fighter_f_name'] + ' ' + fighter_data['fighter_l_name']

# Fazer o merge
ufc_fighter_stats = pd.merge(fight_stat_data, fight_data,
                             left_on='fight_url', 
                             right_on='fight_url', 
                             how='left')

ufc_fighter_stats = pd.merge(ufc_fighter_stats, fighter_data, 
                             left_on='winner', 
                             right_on='full_name', 
                             how='left')

# Salvar o resultado
ufc_fighter_stats.drop_duplicates()

,fighter_id,knockdowns,total_strikes_att,total_strikes_succ,sig_strikes_att,sig_strikes_succ,takedown_att,takedown_succ,submission_att,reversals,...,fighter_weight_lbs,fighter_reach_cm,fighter_stance,fighter_dob,fighter_w,fighter_l,fighter_d,fighter_nc_dq,fighter_url,full_name
0,Marcin Tybura,0.0,18.0,9.0,4.0,0.0,0.0,0.0,0.0,1.0,...,260.0,198.12,Orthodox,1995-01-24,17.0,4.0,0.0,NaN,http://ufcstats.com/fighter-details/e2f6b2769a...,Serghei Spivac
1,Serghei Spivac,0.0,11.0,8.0,5.0,3.0,1.0,1.0,1.0,0.0,...,260.0,198.12,Orthodox,1995-01-24,17.0,4.0,0.0,NaN,http://ufcstats.com/fighter-details/e2f6b2769a...,Serghei Spivac
2,Damon Jackson,0.0,72.0,49.0,49.0,32.0,5.0,1.0,0.0,1.0,...,145.0,175.26,Orthodox,1992-10-27,17.0,6.0,0.0,1.0,http://ufcstats.com/fighter-details/e0c6edcb5b...,Chepe Mariscal
3,Chepe Mariscal,0.0,267.0,218.0,100.0,73.0,7.0,5.0,0.0,2.0,...,145.0,175.26,Orthodox,1992-10-27,17.0,6.0,0.0,1.0,http://ufcstats.com/fighter-details/e0c6edcb5b...,Chepe Mariscal
4,Danny Barlow,0.0,107.0,52.0,99.0,47.0,2.0,0.0,0.0,0.0,...,170.0,200.66,Southpaw,1995-08-02,9.0,0.0,0.0,NaN,http://ufcstats.com/fighter-details/b7c5eb7cdb...,Danny Barlow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15213,David Levicki,0.0,102.0,95.0,5.0,4.0,0.0,0.0,0.0,0.0,...,210.0,NaN,Orthodox,NaN,2.0,1.0,0.0,NaN,http://ufcstats.com/fighter-details/319fa1bd31...,Johnny Rhodes
15214,Patrick Smith,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,225.0,NaN,Orthodox,1963-08-28,20.0,17.0,0.0,NaN,http://ufcstats.com/fighter-details/46c8ec317a...,Patrick Smith
15215,Ray Wizard,0.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,...,225.0,NaN,Orthodox,1963-08-28,20.0,17.0,0.0,NaN,http://ufcstats.com/fighter-details/46c8ec317a...,Patrick Smith
15216,Scott Morris,0.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,...,210.0,NaN,Orthodox,NaN,2.0,1.0,0.0,NaN,http://ufcstats.com/fighter-details/be9d259be0...,Scott Morris


In [352]:
ufc_fighter_stats.columns
ufc_fighter_stats.shape

(15218, 39)

In [361]:
# Criar um DataFrame separado para calcular o momento
momentum_df = ufc_fighter_stats[['fighter_id', 'winner', 'fight_url']].copy()
momentum_df['win'] = momentum_df['fighter_id'] == momentum_df['winner']

# Ordenar o DataFrame por fighter_id e fight_url em ordem ascendente
momentum_df = momentum_df.sort_values(['fighter_id', 'fight_url'])

# Calcular o momento para cada lutador
def calculate_momentum(group):
    wins = group['win'].astype(int).rolling(window=5, min_periods=1).sum()
    return wins.fillna(0)

momentum_df['momentum'] = momentum_df.groupby('fighter_id').apply(lambda x: calculate_momentum(x)).reset_index(level=0, drop=True)

# Juntar o momento calculado ao DataFrame original
ufc_fighter_stats = ufc_fighter_stats.merge(momentum_df[['fight_url', 'fighter_id', 'momentum']], 
                                            on=['fight_url', 'fighter_id'], 
                                            how='left')

ufc_fighter_stats


/var/folders/82/jg5v42nd1m352rr46mcjlyl80000gp/T/ipykernel_63990/3142943104.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  momentum_df['momentum'] = momentum_df.groupby('fighter_id').apply(lambda x: calculate_momentum(x)).reset_index(level=0, drop=True)


,fighter_id,knockdowns,total_strikes_att,total_strikes_succ,sig_strikes_att,sig_strikes_succ,takedown_att,takedown_succ,submission_att,reversals,...,fighter_reach_cm,fighter_stance,fighter_dob,fighter_w,fighter_l,fighter_d,fighter_nc_dq,fighter_url,full_name,momentum
0,Marcin Tybura,0.0,18.0,9.0,4.0,0.0,0.0,0.0,0.0,1.0,...,198.12,Orthodox,1995-01-24,17.0,4.0,0.0,NaN,http://ufcstats.com/fighter-details/e2f6b2769a...,Serghei Spivac,3.0
1,Serghei Spivac,0.0,11.0,8.0,5.0,3.0,1.0,1.0,1.0,0.0,...,198.12,Orthodox,1995-01-24,17.0,4.0,0.0,NaN,http://ufcstats.com/fighter-details/e2f6b2769a...,Serghei Spivac,4.0
2,Damon Jackson,0.0,72.0,49.0,49.0,32.0,5.0,1.0,0.0,1.0,...,175.26,Orthodox,1992-10-27,17.0,6.0,0.0,1.0,http://ufcstats.com/fighter-details/e0c6edcb5b...,Chepe Mariscal,2.0
3,Chepe Mariscal,0.0,267.0,218.0,100.0,73.0,7.0,5.0,0.0,2.0,...,175.26,Orthodox,1992-10-27,17.0,6.0,0.0,1.0,http://ufcstats.com/fighter-details/e0c6edcb5b...,Chepe Mariscal,3.0
4,Danny Barlow,0.0,107.0,52.0,99.0,47.0,2.0,0.0,0.0,0.0,...,200.66,Southpaw,1995-08-02,9.0,0.0,0.0,NaN,http://ufcstats.com/fighter-details/b7c5eb7cdb...,Danny Barlow,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15293,David Levicki,0.0,102.0,95.0,5.0,4.0,0.0,0.0,0.0,0.0,...,NaN,Orthodox,NaN,2.0,1.0,0.0,NaN,http://ufcstats.com/fighter-details/319fa1bd31...,Johnny Rhodes,0.0
15294,Patrick Smith,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,NaN,Orthodox,1963-08-28,20.0,17.0,0.0,NaN,http://ufcstats.com/fighter-details/46c8ec317a...,Patrick Smith,1.0
15295,Ray Wizard,0.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,...,NaN,Orthodox,1963-08-28,20.0,17.0,0.0,NaN,http://ufcstats.com/fighter-details/46c8ec317a...,Patrick Smith,0.0
15296,Scott Morris,0.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,Orthodox,NaN,2.0,1.0,0.0,NaN,http://ufcstats.com/fighter-details/be9d259be0...,Scott Morris,1.0


In [363]:
# Importar bibliotecas necessárias
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd
from datetime import datetime

# Função para obter estatísticas do lutador
def get_fighter_stats(df, fighter_id):
    return df[df['fighter_id'] == fighter_id].iloc[0]

# Função para calcular a idade
def calculate_age(born):
    if pd.isnull(born):
        return np.nan
    born = datetime.strptime(born, "%Y-%m-%d")
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# Preparar os dados para o modelo
features = ['knockdowns', 'total_strikes_att', 'total_strikes_succ', 'sig_strikes_att', 'sig_strikes_succ', 
            'takedown_att', 'takedown_succ', 'submission_att', 'reversals', 'age', 'momentum']

# Adicionar a coluna de idade ao DataFrame original
ufc_fighter_stats['age'] = ufc_fighter_stats['fighter_dob'].apply(calculate_age)

# Criar um novo DataFrame para armazenar os dados de ambos os lutadores
fight_data = []

for _, row in ufc_fighter_stats.iterrows():
    fighter_1_id = row['f_1']
    fighter_2_id = row['f_2']
    
    fighter_1_stats = get_fighter_stats(ufc_fighter_stats, fighter_1_id)
    fighter_2_stats = get_fighter_stats(ufc_fighter_stats, fighter_2_id)
    
    # Criar duas entradas para cada luta, invertendo a ordem dos lutadores
    for order in ['normal', 'invertido']:
        fight_features = {}
        if order == 'normal':
            fighters = [('fighter_1_', fighter_1_stats), ('fighter_2_', fighter_2_stats)]
            winner = 1 if row['winner'] == fighter_1_id else 0
        else:
            fighters = [('fighter_1_', fighter_2_stats), ('fighter_2_', fighter_1_stats)]
            winner = 0 if row['winner'] == fighter_1_id else 1
        
        for prefix, stats in fighters:
            for feature in features:
                fight_features[f'{prefix}{feature}'] = stats[feature]
            
            # Adicionar campos _att e _succ
            for att_succ_feature in ['total_strikes', 'sig_strikes', 'takedown']:
                fight_features[f'{prefix}{att_succ_feature}_att'] = stats[f'{att_succ_feature}_att']
                fight_features[f'{prefix}{att_succ_feature}_succ'] = stats[f'{att_succ_feature}_succ']
            
            # Calcular taxas de efetividade
            for att_feature in ['total_strikes', 'sig_strikes', 'takedown']:
                att = stats[f'{att_feature}_att']
                succ = stats[f'{att_feature}_succ']
                if att != 0:
                    fight_features[f'{prefix}{att_feature}_effectiveness'] = succ / att
                else:
                    fight_features[f'{prefix}{att_feature}_effectiveness'] = 0
        
        # Calcular o win ratio corretamente
        fight_features['fighter_1_win_ratio'] = fighter_1_stats['fighter_w'] / (fighter_1_stats['fighter_w'] + fighter_1_stats['fighter_l']) if (fighter_1_stats['fighter_w'] + fighter_1_stats['fighter_l']) != 0 else 0
        fight_features['fighter_2_win_ratio'] = fighter_2_stats['fighter_w'] / (fighter_2_stats['fighter_w'] + fighter_2_stats['fighter_l']) if (fighter_2_stats['fighter_w'] + fighter_2_stats['fighter_l']) != 0 else 0
        
        fight_features['winner'] = winner
        
        fight_data.append(fight_features)

fight_df = pd.DataFrame(fight_data)

# Remover colunas 'att' e 'succ' após calcular a efetividade
columns_to_drop = [col for col in fight_df.columns if '_att' in col or '_succ' in col]
fight_df = fight_df.drop(columns=columns_to_drop)

# Preparar X e y
X = fight_df.drop('winner', axis=1)
y = fight_df['winner']

# Lidar com valores ausentes
X = X.fillna(X.mean())

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Criar e treinar o modelo
model = XGBClassifier(random_state=42)
model.fit(X_train_scaled, y_train)

# Fazer previsões
y_pred = model.predict(X_test_scaled)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")

print("\nRelatório de classificação:")
print(classification_report(y_test, y_pred))

# Importância das features
feature_importance = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_})
feature_importance = feature_importance.sort_values('importance', ascending=False)


Acurácia do modelo: 0.82

Relatório de classificação:
              precision    recall  f1-score   support

           0       0.82      0.81      0.82      3062
           1       0.81      0.82      0.82      3058

    accuracy                           0.82      6120
   macro avg       0.82      0.82      0.82      6120
weighted avg       0.82      0.82      0.82      6120



In [364]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Solicitar input do usuário
lutador1 = 'Kevin Jousset'
lutador2 = 'Bryan Battle'


# Verificar se os DataFrames necessários já existem
if 'ufc_fighter_stats' not in globals() or 'X' not in globals() or 'y' not in globals() or 'model' not in globals() or 'scaler' not in globals():
    print("Erro: Alguns DataFrames ou modelos necessários não foram encontrados.")
    print("Por favor, execute as células anteriores para criar os DataFrames e treinar o modelo.")
else:
    # Função para obter as estatísticas médias de um lutador específico
    def obter_estatisticas_lutador_v2(nome_lutador):
        lutador_stats = ufc_fighter_stats[ufc_fighter_stats['fighter_id'] == nome_lutador]
        if lutador_stats.empty:
            print(f"Erro: Não foram encontradas estatísticas para o lutador {nome_lutador}")
            return None
        
        # Calcular a média apenas para colunas numéricas, excluindo num_rounds
        colunas_numericas = lutador_stats.select_dtypes(include=[np.number]).columns
        colunas_numericas = [col for col in colunas_numericas if col != 'num_rounds']
        stats_medias = lutador_stats[colunas_numericas].mean()
        
        # Para valores não numéricos, pegamos o último valor
        stats_medias['age'] = lutador_stats['age'].iloc[-1] if not lutador_stats['age'].empty else None
        
        # Calculando o win_ratio corretamente
        total_lutas = stats_medias['fighter_w'] + stats_medias['fighter_l']
        stats_medias['win_ratio'] = stats_medias['fighter_w'] / total_lutas if total_lutas > 0 else 0
        
        # Calculando as novas variáveis de efetividade
        stats_medias['total_strikes_eff'] = stats_medias['total_strikes_succ'] / stats_medias['total_strikes_att'] if stats_medias['total_strikes_att'] > 0 else 0
        stats_medias['sig_strikes_eff'] = stats_medias['sig_strikes_succ'] / stats_medias['sig_strikes_att'] if stats_medias['sig_strikes_att'] > 0 else 0
        stats_medias['takedown_eff'] = stats_medias['takedown_succ'] / stats_medias['takedown_att'] if stats_medias['takedown_att'] > 0 else 0
        
        # Adicionando a média dos campos _att e _succ
        stats_medias['total_strikes_att'] = lutador_stats['total_strikes_att'].mean()
        stats_medias['total_strikes_succ'] = lutador_stats['total_strikes_succ'].mean()
        stats_medias['sig_strikes_att'] = lutador_stats['sig_strikes_att'].mean()
        stats_medias['sig_strikes_succ'] = lutador_stats['sig_strikes_succ'].mean()
        stats_medias['takedown_att'] = lutador_stats['takedown_att'].mean()
        stats_medias['takedown_succ'] = lutador_stats['takedown_succ'].mean()
        
        # Convertendo finish_time para minutos antes de calcular a média
        def time_to_minutes(time_str):
            try:
                minutes, seconds = map(int, time_str.split(':'))
                return minutes + seconds / 60
            except:
                return 0  # Retorna 0 se houver erro na conversão

        lutador_stats['finish_time_minutes'] = lutador_stats['finish_time'].apply(time_to_minutes)
        stats_medias['finish_time'] = lutador_stats['finish_time_minutes'].mean()
        
        # Contagem de vitórias por tipo de resultado
        lutador_vitorias = lutador_stats[lutador_stats['winner'] == nome_lutador]
        vitorias_por_resultado = lutador_vitorias['result'].value_counts().to_dict()
        
        # Adicionando as contagens de vitórias diretamente ao dicionário stats_medias
        for resultado, contagem in vitorias_por_resultado.items():
            stats_medias[f'vitorias_por_{resultado}'] = contagem
        
        # Adicionando total de vitórias
        stats_medias['total_vitorias'] = sum(vitorias_por_resultado.values())
        
        # Adicionando a última linha de momentum
        stats_medias['momentum'] = lutador_stats['momentum'].iloc[-1] if not lutador_stats['momentum'].empty else None
        
        return stats_medias
    # Função para prever o resultado da luta
    def prever_luta_v2(lutador1, lutador2):
        # Obter estatísticas médias para os lutadores
        lutador1_stats = obter_estatisticas_lutador_v2(lutador1)
        lutador2_stats = obter_estatisticas_lutador_v2(lutador2)

        if lutador1_stats is None or lutador2_stats is None:
            print("Não foi possível fazer a previsão devido à falta de dados.")
            return

        # Preparar os dados para previsão
        fight_features = {}
        for prefix, stats in [('fighter_1_', lutador1_stats), ('fighter_2_', lutador2_stats)]:
            for col in X.columns:
                if col.startswith(prefix):
                    feature = col.replace(prefix, '')
                    if feature in stats.index:
                        fight_features[col] = stats[feature]
                    else:
                        fight_features[col] = 0  # ou outro valor padrão apropriado

        fight_df = pd.DataFrame([fight_features])

        # Garantir que todas as colunas do modelo estejam presentes
        for col in X.columns:
            if col not in fight_df.columns:
                fight_df[col] = 0  # ou outro valor padrão apropriado

        # Reordenar as colunas para corresponder ao modelo
        fight_df = fight_df[X.columns]

        # Lidar com valores ausentes
        fight_df = fight_df.fillna(X.mean())

        # Normalizar os dados
        fight_scaled = scaler.transform(fight_df)

        # Fazer previsões
        previsoes = model.predict_proba(fight_scaled)

        # Determinar o vencedor
        vencedor = lutador1 if previsoes[0][1] > previsoes[0][0] else lutador2

        return vencedor

    # Chamar a função para prever o resultado da luta
    vencedor = prever_luta_v2(lutador1, lutador2)
    print(f"O vencedor previsto é: {vencedor}")


O vencedor previsto é: Bryan Battle


In [365]:
import numpy as np

ufc_fight_data = pd.read_csv('/Users/joaosequeira/ufc-stats/UFC-Web-Scraping/ufc_web_scraper/scraped_files/ufc_fight_data.csv')

# Inverter a tabela para ordem crescente
ufc_fight_data = ufc_fight_data.iloc[::-1].reset_index(drop=True)

# Realizar a validação cruzada temporal
n_splits = 5
tamanho_split = len(ufc_fight_data) // n_splits
acuracias = []

for i in range(n_splits):
    resultados_reais = []
    resultados_previstos = []
    
    train_data = ufc_fight_data.iloc[:tamanho_split * (i + 1)]
    test_data = ufc_fight_data.iloc[tamanho_split * (i + 1):tamanho_split * (i + 2)]
    
    for index, row in test_data.iterrows():
        lutador1 = row['f_1']
        lutador2 = row['f_2']
        vencedor_real = row['winner']
        
        # Obter o vencedor previsto usando prever_luta_v2
        vencedor_previsto = prever_luta_v2(lutador1, lutador2)
        
        if vencedor_previsto is None:
            raise TypeError("A função prever_luta_v2 retornou None em vez de um valor válido.")
        
        resultados_reais.append(vencedor_real)
        resultados_previstos.append(vencedor_previsto)
    
    # Calcular a porcentagem de previsão acertada para esta divisão
    acertos = sum([1 for real, previsto in zip(resultados_reais, resultados_previstos) if real == previsto])
    total = len(resultados_reais)
    porcentagem_acerto = (acertos / total)
    acuracias.append(porcentagem_acerto)

# Calcular a média das acurácias
media_acuracia = np.mean(acuracias)
print(f"Média de acurácia após validação cruzada: {media_acuracia:.2f}")

Média de acurácia após validação cruzada: 0.72


In [372]:
# Função para prever o vencedor de uma luta
def prever_vencedor(lutador1, lutador2):
    vencedor = prever_luta_v2(lutador1, lutador2)
    return vencedor

# Lista de lutas futuras
lutas_futuras = [
    ("Israel Adesanya", "Dricus Du Plessis"),
    ("Steve Erceg", "Kai Kara-France"),
    ("Mateusz Gamrot", "Dan Hooker"),
    ("Jair Rozenstruik", "Tai Tuivasa"),
    ("Carlos Prates", "Li Jingliang"),
    ("Junior Tafa", "Valter Walker"),
    ("Josh Culibao", "Ricardo Ramos"),
    ("Luana Santos", "Casey O'Neill"),
    ("Jack Jenkins", "Herbert Burns"),
    ("Tom Nolan", "Alex Reyes"),
    ("Kenan Song", "Rick Glenn"),
    ("Stewart Nicoll", "Jesus Aguilar") 
]

print("Previsões para as próximas lutas:")
print("=================================")

for lutador1, lutador2 in lutas_futuras:
    vencedor = prever_vencedor(lutador1, lutador2)
    print(f"Luta: {lutador1} vs {lutador2}")
    print(f"Vencedor previsto: {vencedor}")
    print("---------------------------------")

print("=================================")
print("Fim das previsões")


Previsões para as próximas lutas:
Luta: Israel Adesanya vs Dricus Du Plessis
Vencedor previsto: Dricus Du Plessis
---------------------------------
Luta: Steve Erceg vs Kai Kara-France
Vencedor previsto: Kai Kara-France
---------------------------------
Luta: Mateusz Gamrot vs Dan Hooker
Vencedor previsto: Mateusz Gamrot
---------------------------------
Erro: Não foram encontradas estatísticas para o lutador Jair Rozenstruik
Não foi possível fazer a previsão devido à falta de dados.
Luta: Jair Rozenstruik vs Tai Tuivasa
Vencedor previsto: None
---------------------------------
Luta: Carlos Prates vs Li Jingliang
Vencedor previsto: Carlos Prates
---------------------------------
Luta: Junior Tafa vs Valter Walker
Vencedor previsto: Valter Walker
---------------------------------
Luta: Josh Culibao vs Ricardo Ramos
Vencedor previsto: Ricardo Ramos
---------------------------------
Luta: Luana Santos vs Casey O'Neill
Vencedor previsto: Luana Santos
---------------------------------
Luta: